# ⚖️ Legal-RAG: Kat Mülkiyeti Kanunu Asistanı

## 1. Proje Tanıtımı
**Amaç:** Kullanıcıların Kat Mülkiyeti Kanunu ile ilgili sorularını, **doğrudan kanun metnine dayanarak** cevaplayan yapay zeka destekli bir asistan geliştirmek.

**Hedef Kitle:** Apartman yöneticileri, kat malikleri ve hukuki bilgiye hızlı erişmek isteyenler.

**Temel Sorun:** Hukuki metinlerin karmaşık dili ve aradığını bulmanın zorluğu.
**Çözüm:** RAG (Retrieval-Augmented Generation) mimarisi ile doğal dilde soru-cevap.

## 2. Mimari ve Çalışma Mantığı

Projemiz modern bir **RAG** mimarisi kullanır. Şöyle çalışır:

1.  **Ingestion (Veri Hazırlığı):** PDF okunur, parçalanır ve vektörlere dönüştürülür.
2.  **Storage (Saklama):** Vektörler `ChromaDB` veritabanında saklanır.
3.  **Retrieval (Getirme):** Kullanıcı sorusuyla en alakalı parçalar bulunur.
4.  **Generation (Üretim):** Bulunan parçalar ve soru LLM'e (Gemini) gönderilir, cevap üretilir.

## 3. Karşılaşılan Zorluklar ve Çözümler

Bu projeyi geliştirirken yaşadığımız kritik anlar:

### 🛑 Zorluk 1: Halüsinasyon 
**Sorun:** LLM bazen kanunda olmayan şeyleri uydurabiliyordu.
**Çözüm:** System Prompt'u çok sıkı tuttuk: *"Asla uydurma, sadece verilen metni kullan, madde numarası ver."*

### 🛑 Zorluk 2: Token Limitleri ve Bağlam
**Sorun:** Tüm kanunu tek seferde modele gönderemeyiz.
**Çözüm:** **Chunking** stratejisi. Metni 2000 karakterlik mantıksal parçalara böldük. Ayrıca metnin anlam bütünlüğü bozulmasın diye 400 karakterlik **overlap** (örtüşme) payı bıraktık.

### 🛑 Zorluk 3: API Kotaları ve Hız
**Sorun:** Her çalıştırmada tekrar vektör oluşturmak (Embedding) çok zaman alıyor ve API kotasını bitiriyor.
**Çözüm:** **Vector Persistence (Kalıcılık).** Veritabanını diske (`chroma_db` klasörüne) kaydettik. Uygulama açılırken önce diske bakıyor, varsa oradan yüklüyor.

## 4. Kod Üzerinden Anlatım

Şimdi sistemin parçalarını canlı olarak inceleyelim. Kodlarımız `src/` klasörü altında modüler bir yapıda.

In [9]:
# Gerekli kütüphaneleri ve modülleri yükleyelim
import sys
import os
from dotenv import load_dotenv

# Proje kök dizinini yola ekle (src modülünü bulabilmesi için)
sys.path.append(os.path.abspath('.'))

# Ortam değişkenlerini yükle (API KEY)
load_dotenv()

from src import config
from src.ingestion import get_vectorstore
from src.rag import LegalRAG

print(f"Mode: {config.LLM_MODEL_NAME}")
print(f"Database Path: {config.CHROMA_PERSIST_DIRECTORY}")

Mode: gemini-2.5-flash
Database Path: /home/tumay/code/kat-mulkiyeti-legal-rag-research/chroma_db


### Adım 1: Veritabanına Bağlanma
Burada `get_vectorstore` fonksiyonumuz akıllı çalışır. Eğer veritabanı daha önce oluşturulmuşsa, tekrar PDF okumaz, direkt diskten yükler.

In [10]:
# Veritabanını yükle (Sade ve Hızlı)
vectorstore = get_vectorstore(force_recreate=False)

# Koleksiyondaki toplam parça sayısına bakalım
count = vectorstore._collection.count()
print(f"Veritabanındaki toplam chunk sayısı: {count}")

Mevcut vektör veritabanı bulundu: /home/tumay/code/kat-mulkiyeti-legal-rag-research/chroma_db
Veritabanı başarıyla yüklendi.
Veritabanındaki toplam chunk sayısı: 67


### Adım 2: RAG Motorunu Başlatma
Retrieval (Arama) ve Generation (Üretim) işlemlerini yöneten sınıfımız.

In [11]:
rag_system = LegalRAG(vectorstore)

LLM (gemini-2.5-flash) hazırlanıyor...
LLM hazır.


### Adım 3: Canlı Test
Hocamızın veya proje yöneticimizin sorabileceği örnek bir soruyu soralım.

In [12]:
soru = "Kira artış oranları hangi ayda açıklanır?"

# 1. Retrieval (Arama): Önce en alakalı parçaları bulalım
docs = rag_system.get_relevant_docs(soru)

print("🔍 Bulunan Alakalı Doküman Parçaları:\n")
for i, doc in enumerate(docs[:2]): # İlk 2'sini göster
    print(f"--- Parça {i+1} ---")
    print(doc.page_content[:200] + "...") # İlk 200 karakter
    print("\n")

🔍 Bulunan Alakalı Doküman Parçaları:

--- Parça 1 ---
KAT MÜLKİYETİ KANUNU 
 
Kanun Numarası : 634 
Kabul Tarihi : 23/6/1965 
Yayımlandğı Resmî Gazete : Tarih  : 2/7/1965 Sayı : 12038 
Yayımlandığı Düstur : Tertip : 5 Cilt : 4 Sayfa : 2932 
 
BİRİNCİ BÖL...


--- Parça 2 ---
Madde 4 
 
14/4/1983 
3227 Sekizinci Bölüm, 57, 58, 59, 
60, 61, 62, 63, 64, 65   25/6/1985 
KHK/431 42 13/2/1991 
3770 42 11/2/1992 
5378 42 7/7/2005 
5627 42 2/5/2007 
5711 
3, 4, 10, 11, 12, 13, 14...




In [13]:
# 2. Generation (Cevap Üretme): Gemini'ye gönderip cevap alalım
result = rag_system.get_answer(soru)

print("ASİSTAN CEVABI:\n")
print(result['answer'])

ASİSTAN CEVABI:

Bu konuda verilen metinlerde bilgi bulunmamaktadır.


## 5. Future Work

Bu projeyi daha da geliştirmek için yapacaklarımız:

1.  **Gelişmiş Arayüz:** Streamlit'e sohbet geçmişi (Chat History) eklenebilir.
2.  **Çoklu Kaynak:** Sadece kanun değil, ilgili Yargıtay kararlarını da eklenebilir.
3.  **Agentlar:** Sadece cevaplayan değil, internetten emsal karar araştıran "Research Agent" eklenebilir.